* 페이지 진입 > 박스오피스 메뉴
 일별(default)
  - 24년 9/13 start_date 설정하여 일주일치 한번에 
  - 25년 9/26, 10/3, 10/10, 10/17 start_date 설정하여 일주일치 한번에 
  - 각 지역별로 조회
  - 한국/외국 나눠서
 ~~월별 메뉴 이동~~
 ~~연도별 메뉴 이동~~


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
import time
import random
from tqdm import tqdm


In [12]:
def set_daily_searh_date_option_by_start_date(driver, start_date):
    # global driver
    # 2.1 시작일 선택
    driver.find_element(By.ID, "sSearchFrom").click()
    # 2.2 년 
    wanna_this_year = start_date[0]#"2025"
    y_select = driver.find_element(By.CSS_SELECTOR,".ui-datepicker-year")
    Select(y_select).select_by_value(wanna_this_year)
    # 2.3 월
    wanna_this_month = start_date[1]#"9월"
    m_select = driver.find_element(By.CSS_SELECTOR,".ui-datepicker-month")
    Select(m_select).select_by_visible_text(wanna_this_month)
    # 2.4 일
    wanna_this_day = start_date[2]#"26"
    d_table = driver.find_element(By.CSS_SELECTOR,".ui-datepicker-calendar")
    d_elems = d_table.find_elements(By.TAG_NAME, "a")
    for d_elem in d_elems:
        # print(d_elem.text)#클릭 이후 찍으면 -> 에러! 왜냐? 알아봐라. 대충은 알겠음.
        if d_elem.text == wanna_this_day:
            # print("! ", d_elem.text)#클릭 이후 찍으면 -> 에러! 왜냐? 알아봐라. 대충은 알겠음.
            d_elem.click()
            break


In [13]:
from selenium.common.exceptions import StaleElementReferenceException

# 조회 클릭
def clickSearchButton():
    print('조회!')
    global driver
    
    try:
        search_btn = driver.find_element(By.CLASS_NAME, "wrap_btn").find_element(By.TAG_NAME, "button")
        WebDriverWait(driver, 240).until(EC.element_to_be_clickable(search_btn))
        search_btn.click()
        
    except StaleElementReferenceException:
        print("StaleElementReferenceException!")
        time.sleep(1)
        search_btn = driver.find_element(By.CLASS_NAME, "wrap_btn").find_element(By.TAG_NAME, "button")
        WebDriverWait(driver, 240).until(EC.element_to_be_clickable(search_btn))
        search_btn.click()

    except TimeoutError as e:
        print("TimeoutException!")
        # time.sleep(60)# 서버에서 짤리면 의미없음. 그리고, 에러 여기로 잡히지 않는 것 같음.
        
    finally:
        pass


    

In [14]:
from httpcore import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoAlertPresentException

def downloadExcelFile():
    print('Excel 다운로드!')
    global driver

    try:
        exl_btn = driver.find_element(By.CSS_SELECTOR,".f_sch").find_element(By.TAG_NAME, "a")
        WebDriverWait(driver, 240).until(EC.element_to_be_clickable(exl_btn))
        exl_btn.click()
    except StaleElementReferenceException:
        print("StaleElementReferenceException!")
        time.sleep(1)
        exl_btn = driver.find_element(By.CSS_SELECTOR,".f_sch").find_element(By.TAG_NAME, "a")
        WebDriverWait(driver, 240).until(EC.element_to_be_clickable(exl_btn))
        exl_btn.click()
    except TimeoutException as e:
        print("TimeoutException!")
        time.sleep(60)
    finally:
        pass

    try:
        alert = driver.switch_to.alert
        alert.accept()
        print("clearExlDownAlert")
    except NoAlertPresentException as e:
        print('NoAlertPresentException!')#, str(e))


In [15]:
# 박스오피스 일별 진입

options = webdriver.ChromeOptions()
download_path = '/home/addinedu/Project_ws/eda-repo-2/Boxoffice/output'
driver_path = '/home/addinedu/Project_ws/eda-repo-2/ShoppingList/driver/chromedriver'
prefs = {'download.default_directory':download_path, 'download.prompt_for_download':False}
options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(service=Service(driver_path), options=options)
driver.get("https://www.kobis.or.kr/")

# Top Menu : boxoffice click
top_menu_link_list = driver.find_elements(By.CLASS_NAME, 'tit_nav')
box_office_link = top_menu_link_list[1]
box_office_link.send_keys(Keys.ENTER)

In [16]:
#   - 24년 9/13 start_date 설정하여 일주일치 한번에 
#   - 25년 9/26, 10/3, 10/10, 10/17 start_date 설정하여 일주일치 한번에 
daily_start_date_list = [
    ['2024', '9월', '13'],
    ['2025', '9월', '26'],
    ['2025', '10월', '3'],
    ['2025', '10월', '10'],
    ['2025', '10월', '17']
]

#한국/외국
nation_select = driver.find_element(By.ID, "sRepNationCd")
nation_option_list = nation_select.find_elements(By.TAG_NAME, "option")
nation_list = [n_opt.get_attribute("value") for n_opt in nation_option_list if n_opt.get_attribute("value")]
print('nation_list', nation_list)

#region
#지역 선택: 개별지역 돌아가며
region_select = driver.find_element(By.ID, "sWideAreaCd")
region_option_list = region_select.find_elements(By.TAG_NAME, "option")
region_list = [r_opt.get_attribute("value") for r_opt in region_option_list if r_opt.get_attribute("value")]

#date
date_len = len(daily_start_date_list)
# for di, start_date in enumerate(tqdm(daily_start_date_list)):
for di, start_date in enumerate(daily_start_date_list):
    print(di+1, '/', date_len, start_date, end=' ')
    set_daily_searh_date_option_by_start_date(driver, start_date)
    time.sleep(0.5)

    for n_val in nation_list:
        print(n_val, end='')
        nation_select = driver.find_element(By.ID, "sRepNationCd")
        Select(nation_select).select_by_value(n_val)

        for r_val in tqdm(region_list):#[:3]):#!! 일부만!!
            # print(r_val)
            region_select = driver.find_element(By.ID, "sWideAreaCd")# !!이렇게 안쓰니 StaleElementReferenceException 에러가 나는데...
            Select(region_select).select_by_value(r_val)
            time.sleep( random.uniform(1,2) )

            clickSearchButton() # type: ignore # time.sleep( random.uniform(5,8) )

            downloadExcelFile()
            time.sleep( random.uniform(3,5) )


nation_list ['K', 'F']
1 / 5 ['2024', '9월', '13'] K

  0%|          | 0/17 [00:00<?, ?it/s]

조회!
Excel 다운로드!
clearExlDownAlert


  6%|▌         | 1/17 [00:07<01:57,  7.33s/it]

조회!
Excel 다운로드!
clearExlDownAlert


 12%|█▏        | 2/17 [00:14<01:45,  7.02s/it]


KeyboardInterrupt: 

^^^^ KeyboardInterrupt : 다 받은 후 재시도. 로그 줄이기 위해 강제 종료시킴 ^^^^

In [19]:
# down file check. len
from glob import glob

file_list = glob(download_path + '/KOBIS_*')
len(file_list)


170

In [20]:
import pandas as pd
import re

def get_dataframe_from_daily_excel(soup):
    div_elems = soup.find_all('div')
    table_elems = soup.find_all('table')[1:]#날짜별로, 첫 테이블은 검색조건

    total_data_list = []

    for ti, table in enumerate(table_elems):
        date_div = div_elems[ti]
        date = re.findall(r'\d+년 \d+월 \d+일', date_div.text.strip())[0]# print(date_div.text.strip())

        rows = table.find_all('tr')
        end_row_num = len(rows)
        for ri,row in enumerate(rows):

            row_data = []

            if ri==0:#head
                if ti == 0:
                    row_data.append('일자')
                    cols = row.find_all('th')
                    for ci, th in enumerate(cols):
                        if ci in (1,2,3,8,12,13,14,16,18,19,20,21):#영화명, 개봉일, 매출액, 관객수, 스크린수, 상영횟수, 14대표국적, 16제작사, 17배급사, 18등급, 19장르, 20감독, 21배우
                            row_data.append(th.text.strip())

                    total_data_list.append(row_data)

            elif ri == end_row_num-1:#bottom sum
                pass

            else:
                row_data.append(date)
                cols = row.find_all('td')

                for ci, td in enumerate(cols):
                    # if ci == 0:
                    #     row_data.append(int(td.text.strip()))
                    # elif ci == 1:#str 영화명
                    #     row_data.append(td.text.strip())
                    # elif ci == 3:
                    #     row_data.append(int(td.text.strip()))
                    # elif ci == 8:
                    #     row_data.append(int(td.text.strip()))
                    # elif ci == 12:
                    #     row_data.append(int(td.text.strip()))
                    # elif ci == 13:
                    #     row_data.append(int(td.text.strip()))
                    if ci in (1, 14,16,18,19,20,21):
                        row_data.append(td.text.strip())
                    elif ci == 2:
                        row_data.append(td.text.strip())
                    elif ci in (3,8,12,13):
                        row_data.append(int(td.text.strip()))



                total_data_list.append(row_data)

    df = pd.DataFrame(total_data_list[1:], columns=total_data_list[0])

    return df

In [21]:
from bs4 import BeautifulSoup as bs
import pandas as pd

total_df = pd.DataFrame()

for idx, file in enumerate(tqdm(file_list)):
    print(idx, file, end=", ")

    with open(file, 'r') as html:
        html_doc = html.read()
        soup = bs(html_doc,'html.parser')

        # 지역명 확인
        search_option_str = soup.find_all('table')[0].text
        temp_str = re.compile("지역:+.[가-힣]+").search(search_option_str)
        region_name = temp_str.group().replace('지역:','').strip()
        
        print(region_name)#, end=", ")

        df = get_dataframe_from_daily_excel(soup)
        # 지역 col 추가: 모든 열에 같은 값으로
        df['지역'] = region_name
        
        total_df = pd.concat([total_df, df])



  0%|          | 0/170 [00:00<?, ?it/s]

0 /home/addinedu/Project_ws/eda-repo-2/Boxoffice/output/KOBIS_일별_박스오피스_2025-10-22 - 2025-10-22T192958.218.xls, 

  1%|          | 1/170 [00:00<00:48,  3.52it/s]

서울시
1 /home/addinedu/Project_ws/eda-repo-2/Boxoffice/output/KOBIS_일별_박스오피스_2025-10-22 - 2025-10-22T192604.711.xls, 

  2%|▏         | 3/170 [00:00<00:30,  5.53it/s]

...

세종시
166 /home/addinedu/Project_ws/eda-repo-2/Boxoffice/output/KOBIS_일별_박스오피스_2025-10-22 (45).xls, 대구시
167 /home/addinedu/Project_ws/eda-repo-2/Boxoffice/output/KOBIS_일별_박스오피스_2025-10-22 - 2025-10-22T192822.160.xls, 

 99%|█████████▉| 169/170 [00:30<00:00,  6.30it/s]

전라남도
168 /home/addinedu/Project_ws/eda-repo-2/Boxoffice/output/KOBIS_일별_박스오피스_2025-10-22 (10).xls, 부산시
169 /home/addinedu/Project_ws/eda-repo-2/Boxoffice/output/KOBIS_일별_박스오피스_2025-10-22 (63).xls, 

100%|██████████| 170/170 [00:30<00:00,  5.52it/s]

대전시


In [22]:
total_df

,일자,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,제작사,등급,장르,감독,배우,지역
0,2025년 10월 21일,극장판 체인소 맨: 레제편,2025-09-24,83787060,7650,134,493,일본,,15세이상관람가,"애니메이션,액션,어드벤처",,,서울시
1,2025년 10월 21일,원 배틀 애프터 어나더,2025-10-01,20112040,1931,113,217,미국,,15세이상관람가,"범죄,액션",폴 토마스 앤더슨,"레오나르도 디카프리오,숀 펜,베니시오 델 토로,레지나 홀,테야나 테일러,체이스 인피니티",서울시
2,2025년 10월 21일,극장판 귀멸의 칼날: 무한성편,2025-08-22,14885500,1405,78,129,일본,,15세이상관람가,애니메이션,소토자키 하루오,"하나에 나츠키,키토 아카리,시모노 히로,마츠오카 요시츠구,오카모토 노부히코,사쿠라이...",서울시
3,2025년 10월 21일,극장판 주술회전: 회옥·옥절,2025-10-16,13826100,1320,99,209,일본,,15세이상관람가,애니메이션,고쇼조노 쇼타,,서울시
4,2025년 10월 21일,웨폰,2025-10-15,12750600,1286,85,152,미국,,청소년관람불가,"공포(호러),미스터리",잭 크레거,"조슈 브롤린,줄리아 가너,올든 에런라이크,오스틴 아브람스,크리스토퍼 캐리,베네딕트 ...",서울시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,2025년 09월 26일,결혼 피로연,2025-09-24,21000,3,1,2,미국,,15세이상관람가,"코미디,멜로/로맨스",앤드류 안,"윤여정,한기찬,보웬 양,릴리 글래드스톤,켈리 마리 트란,조안 첸",대전시
163,2025년 09월 26일,베르메르: 위대한 전시회,2025-09-17,8000,1,1,1,영국,,전체관람가,다큐멘터리,데이비드 비커스태프,,대전시
164,2025년 09월 26일,부재,2025-08-27,8000,1,1,1,중국,,12세이상관람가,드라마,,,대전시
165,2025년 09월 26일,내 말 좀 들어줘,2025-08-20,10000,1,1,1,영국,,12세이상관람가,드라마,마이크 리,마리안 장 밥티스트,대전시


In [23]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21216 entries, 0 to 166
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      21216 non-null  object
 1   영화명     21216 non-null  object
 2   개봉일     21216 non-null  object
 3   매출액     21216 non-null  int64 
 4   관객수     21216 non-null  int64 
 5   스크린수    21216 non-null  int64 
 6   상영횟수    21216 non-null  int64 
 7   대표국적    21216 non-null  object
 8   제작사     21216 non-null  object
 9   등급      21216 non-null  object
 10  장르      21216 non-null  object
 11  감독      21216 non-null  object
 12  배우      21216 non-null  object
 13  지역      21216 non-null  object
dtypes: int64(4), object(10)
memory usage: 2.4+ MB


## 데이터 전처리
1. datetime: 일자(관객이 영화본날), 개봉일
 - 일자 형식  : 2025년 10월 21일
 - 개봉일 형식 : 2025-09-24
2. int : 매출액, 관객수, 스크린수, 상영횟수
3. object(문자열) : 그외 컬럼

In [24]:
from datetime import datetime

date = datetime.strptime('2024-01-20', '%Y-%m-%d')
date


datetime.datetime(2024, 1, 20, 0, 0)

In [25]:
import pandas as pd

# 형식: "%Y-%m-%d"
total_df['개봉일'] = pd.to_datetime(total_df['개봉일'])
total_df['개봉일']

0     2025-09-24
1     2025-10-01
2     2025-08-22
3     2025-10-16
4     2025-10-15
         ...    
162   2025-09-24
163   2025-09-17
164   2025-08-27
165   2025-08-20
166   2025-09-03
Name: 개봉일, Length: 21216, dtype: datetime64[ns]

In [26]:
import pandas as pd

'''
# 형식: "%Y년 %m월 %d일"
# total_df['일자'] = pd.to_datetime(total_df['일자'])
# total_df['일자']

'raise' (기본값)	변환할 수 없는 값이 있으면 ValueError와 같은 오류를 발생시킵니다.
'coerce'	변환할 수 없는 값을 NaT (Not a Time) 값으로 바꿉니다.
'ignore'	변환할 수 없는 값이 있으면 해당 값을 그대로 반환합니다.
'''

total_df['일자'] = pd.to_datetime(total_df['일자'], format="%Y년 %m월 %d일", errors='coerce')
total_df['일자']

0     2025-10-21
1     2025-10-21
2     2025-10-21
3     2025-10-21
4     2025-10-21
         ...    
162   2025-09-26
163   2025-09-26
164   2025-09-26
165   2025-09-26
166   2025-09-26
Name: 일자, Length: 21216, dtype: datetime64[ns]

In [27]:
total_df['일자'].unique()

<DatetimeArray>
['2025-10-21 00:00:00', '2025-10-20 00:00:00', '2025-10-19 00:00:00',
 '2025-10-18 00:00:00', '2025-10-17 00:00:00', '2025-10-16 00:00:00',
 '2025-10-15 00:00:00', '2025-10-14 00:00:00', '2025-10-13 00:00:00',
 '2025-10-12 00:00:00', '2025-10-11 00:00:00', '2025-10-10 00:00:00',
 '2024-09-19 00:00:00', '2024-09-18 00:00:00', '2024-09-17 00:00:00',
 '2024-09-16 00:00:00', '2024-09-15 00:00:00', '2024-09-14 00:00:00',
 '2024-09-13 00:00:00', '2025-10-09 00:00:00', '2025-10-08 00:00:00',
 '2025-10-07 00:00:00', '2025-10-06 00:00:00', '2025-10-05 00:00:00',
 '2025-10-04 00:00:00', '2025-10-03 00:00:00', '2025-10-02 00:00:00',
 '2025-10-01 00:00:00', '2025-09-30 00:00:00', '2025-09-29 00:00:00',
 '2025-09-28 00:00:00', '2025-09-27 00:00:00', '2025-09-26 00:00:00']
Length: 33, dtype: datetime64[ns]

### 날짜가 뒤죽박죽이다. 정렬을 해보자.

In [28]:
# Sorting: 일자 asc, 지역 asc
total_df = total_df.sort_values(by=['일자', '지역', '매출액'], ascending=[True, True, True])
total_df

,일자,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,제작사,등급,장르,감독,배우,지역
115,2024-09-13,죽고 싶지만 사랑은 하고 싶어,2024-09-04,6000,1,1,1,미국,,12세이상관람가,"드라마,멜로/로맨스",레이첼 램버트,"데이지 리들리,데이브 메르헤예",강원도
116,2024-09-13,래시: 뉴 어드벤처,2024-09-12,6000,1,1,1,독일,,전체관람가,"어드벤처,가족,드라마",,"니코 마리츠카,카타리나 슈틀러",강원도
117,2024-09-13,극장총집편 봇치 더 록! 전편,2024-08-07,13000,1,1,1,일본,,12세이상관람가,애니메이션,사이토 케이이치로,"아오야마 요시노,스즈시로 사유미,미즈노 사쿠,하세가와 이쿠미",강원도
101,2024-09-13,늘봄가든,2024-08-21,14000,2,1,1,한국,(주)바이어스이엔티,15세이상관람가,"공포(호러),스릴러",구태진,"조윤희,이지오,김주령,허동원,정인겸",강원도
114,2024-09-13,주토피아,2016-02-17,18000,2,1,1,미국,,전체관람가,"애니메이션,액션,어드벤처","바이론 하워드,리치 무어","지니퍼 굿윈,제이슨 베이트먼,샤키라",강원도
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,2025-10-21,극장판 귀멸의 칼날: 무한성편,2025-08-22,1194360,117,16,22,일본,,15세이상관람가,애니메이션,소토자키 하루오,"하나에 나츠키,키토 아카리,시모노 히로,마츠오카 요시츠구,오카모토 노부히코,사쿠라이...",충청북도
2,2025-10-21,어쩔수가없다,2025-09-24,1828700,211,24,52,한국,"모호필름,씨제이이엔엠 스튜디오스 주식회사",15세이상관람가,"스릴러,코미디",박찬욱,"이병헌,손예진,박희순,이성민,염혜란,차승원,유연석,이진결",충청북도
1,2025-10-21,연의 편지,2025-10-01,2104800,304,13,14,한국,"스튜디오리코 주식회사,스튜디오엔(주)",12세이상관람가,애니메이션,김용환,"이수현,김민주,민승우,남도형",충청북도
0,2025-10-21,보스,2025-10-03,3714400,510,28,84,한국,(주)하이브미디어코프,15세이상관람가,"코미디,액션",라희찬,"조우진,정경호,박지환,이규형,오달수,황우슬혜,정유진,고창석,이성민",충청북도


In [30]:
# 정렬했으니, reindexing
total_df.reset_index(drop=True, inplace=True)

In [31]:
total_df

,일자,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,제작사,등급,장르,감독,배우,지역
0,2024-09-13,죽고 싶지만 사랑은 하고 싶어,2024-09-04,6000,1,1,1,미국,,12세이상관람가,"드라마,멜로/로맨스",레이첼 램버트,"데이지 리들리,데이브 메르헤예",강원도
1,2024-09-13,래시: 뉴 어드벤처,2024-09-12,6000,1,1,1,독일,,전체관람가,"어드벤처,가족,드라마",,"니코 마리츠카,카타리나 슈틀러",강원도
2,2024-09-13,극장총집편 봇치 더 록! 전편,2024-08-07,13000,1,1,1,일본,,12세이상관람가,애니메이션,사이토 케이이치로,"아오야마 요시노,스즈시로 사유미,미즈노 사쿠,하세가와 이쿠미",강원도
3,2024-09-13,늘봄가든,2024-08-21,14000,2,1,1,한국,(주)바이어스이엔티,15세이상관람가,"공포(호러),스릴러",구태진,"조윤희,이지오,김주령,허동원,정인겸",강원도
4,2024-09-13,주토피아,2016-02-17,18000,2,1,1,미국,,전체관람가,"애니메이션,액션,어드벤처","바이론 하워드,리치 무어","지니퍼 굿윈,제이슨 베이트먼,샤키라",강원도
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21211,2025-10-21,극장판 귀멸의 칼날: 무한성편,2025-08-22,1194360,117,16,22,일본,,15세이상관람가,애니메이션,소토자키 하루오,"하나에 나츠키,키토 아카리,시모노 히로,마츠오카 요시츠구,오카모토 노부히코,사쿠라이...",충청북도
21212,2025-10-21,어쩔수가없다,2025-09-24,1828700,211,24,52,한국,"모호필름,씨제이이엔엠 스튜디오스 주식회사",15세이상관람가,"스릴러,코미디",박찬욱,"이병헌,손예진,박희순,이성민,염혜란,차승원,유연석,이진결",충청북도
21213,2025-10-21,연의 편지,2025-10-01,2104800,304,13,14,한국,"스튜디오리코 주식회사,스튜디오엔(주)",12세이상관람가,애니메이션,김용환,"이수현,김민주,민승우,남도형",충청북도
21214,2025-10-21,보스,2025-10-03,3714400,510,28,84,한국,(주)하이브미디어코프,15세이상관람가,"코미디,액션",라희찬,"조우진,정경호,박지환,이규형,오달수,황우슬혜,정유진,고창석,이성민",충청북도


In [33]:
# to CSV file
total_df.to_csv(download_path + '/kobis_daily_2024_2025_chuseok.csv', sep="|", encoding='utf-8')

In [34]:
# to json file
total_df.to_json(download_path + '/kobis_daily_2024_2025_chuseok.json', indent=2, force_ascii=False)

## !! daily OK !!

### 이제 DB에 데이터 입력한다!

In [39]:
total_df['대표국적'].unique()
len(total_df['대표국적'].unique())

30

In [40]:
#column 순서 정리해야
# 지역 column -> 일자 뒤로 
total_df.columns
total_df = total_df[['일자', '지역', '영화명', '개봉일', '관객수', '스크린수', '상영횟수', '매출액', '대표국적', '제작사', '등급',
       '장르', '감독', '배우']]

### Table 정규화. movie는...
1. boxoffice : ['관람일자', '영화명', '개봉일', '매출액', '관객수', '스크린수', '상영횟수'] #일자 -> 관람일자
2. movie : ['영화명', '개봉일', '대표국적', '제작사', '등급', '장르', '감독', '배우']

In [ ]:
import numpy as np

## total_df is_korean 컬럼 추가 : 대표국적 컬럼 이용해서
total_df['is_korean'] = np.where(total_df['대표국적'] == '한국', 'k', 'f')
total_df

,일자,지역,영화명,개봉일,관객수,스크린수,상영횟수,매출액,대표국적,제작사,등급,장르,감독,배우,is_korean
0,2024-09-13,강원도,죽고 싶지만 사랑은 하고 싶어,2024-09-04,1,1,1,6000,미국,,12세이상관람가,"드라마,멜로/로맨스",레이첼 램버트,"데이지 리들리,데이브 메르헤예",f
1,2024-09-13,강원도,래시: 뉴 어드벤처,2024-09-12,1,1,1,6000,독일,,전체관람가,"어드벤처,가족,드라마",,"니코 마리츠카,카타리나 슈틀러",f
2,2024-09-13,강원도,극장총집편 봇치 더 록! 전편,2024-08-07,1,1,1,13000,일본,,12세이상관람가,애니메이션,사이토 케이이치로,"아오야마 요시노,스즈시로 사유미,미즈노 사쿠,하세가와 이쿠미",f
3,2024-09-13,강원도,늘봄가든,2024-08-21,2,1,1,14000,한국,(주)바이어스이엔티,15세이상관람가,"공포(호러),스릴러",구태진,"조윤희,이지오,김주령,허동원,정인겸",k
4,2024-09-13,강원도,주토피아,2016-02-17,2,1,1,18000,미국,,전체관람가,"애니메이션,액션,어드벤처","바이론 하워드,리치 무어","지니퍼 굿윈,제이슨 베이트먼,샤키라",f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21211,2025-10-21,충청북도,극장판 귀멸의 칼날: 무한성편,2025-08-22,117,16,22,1194360,일본,,15세이상관람가,애니메이션,소토자키 하루오,"하나에 나츠키,키토 아카리,시모노 히로,마츠오카 요시츠구,오카모토 노부히코,사쿠라이...",f
21212,2025-10-21,충청북도,어쩔수가없다,2025-09-24,211,24,52,1828700,한국,"모호필름,씨제이이엔엠 스튜디오스 주식회사",15세이상관람가,"스릴러,코미디",박찬욱,"이병헌,손예진,박희순,이성민,염혜란,차승원,유연석,이진결",k
21213,2025-10-21,충청북도,연의 편지,2025-10-01,304,13,14,2104800,한국,"스튜디오리코 주식회사,스튜디오엔(주)",12세이상관람가,애니메이션,김용환,"이수현,김민주,민승우,남도형",k
21214,2025-10-21,충청북도,보스,2025-10-03,510,28,84,3714400,한국,(주)하이브미디어코프,15세이상관람가,"코미디,액션",라희찬,"조우진,정경호,박지환,이규형,오달수,황우슬혜,정유진,고창석,이성민",k


In [46]:
# dt_id 추가 
# 형식 yymmdd99
total_df['dt_id'] = total_df['일자'].dt.strftime('%y%m%d99')
total_df['dt_id'].unique()


array(['24091399', '24091499', '24091599', '24091699', '24091799',
       '24091899', '24091999', '25092699', '25092799', '25092899',
       '25092999', '25093099', '25100199', '25100299', '25100399',
       '25100499', '25100599', '25100699', '25100799', '25100899',
       '25100999', '25101099', '25101199', '25101299', '25101399',
       '25101499', '25101599', '25101699', '25101799', '25101899',
       '25101999', '25102099', '25102199'], dtype=object)

In [45]:
# total_df.columns.unique()
total_df['is_korean'].unique()

array(['f', 'k'], dtype=object)

In [ ]:
import mysql.connector
import pandas as pd
from mysql.connector import Error

try:
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='1234',
        database='amrbase'
    )
    cursor = conn.cursor()

    for i, row in total_df.iterrows():
        if i == 0:
            continue

        print(i, row)#, end='')

        cursor.execute("""
            INSERT INTO boxoffice (
                            movie_name, release_date, dt_id, region
                            , income, views, screen, play_count, is_korean
                       )                
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """, (
            row['영화명'], row['개봉일'], row['dt_id'], row['지역']
            , row['매출액'], row['관객수'], row['스크린수'], row['상영횟수']
            , row['is_korean']
        ))
    
    conn.commit()
    print("MySQL로 데이터 업로드 OK!")

except Error as e:
    print(f"Error ! {e}")
    pass

finally:
    conn.close()



<<<------------여기까지 Daily boxoffice crawling 완료 ------------>>> 

In [20]:
# left menu : 트리구조로 하위 메뉴들이 있다.
# '월별' 클릭 ('연도별', '일별')
def left_date_menu_change(sub_menu_name):
    '''
    sub_menu_name:'연도별', '월별', '일별'
    '''
    global driver

    left_menu = driver.find_element(By.CLASS_NAME, "list_snb")
    box_office_li = left_menu.find_element(By.CSS_SELECTOR, "li")
    sub_list = box_office_li.find_elements(By.CSS_SELECTOR, "a")
    for a in sub_list:
        # print(a.text)
        if a.text == sub_menu_name:
            a.click()
            break

In [62]:
left_date_menu_change('월별')

In [58]:
def set_monthly_searh_date_option_by_start_date(from_to):
    '''
    from_to = ['2025','09', '2025', '12']
    '''
    global driver

    # 2.1 st 년 
    Select(driver.find_element(By.ID,"sSearchYearFrom")).select_by_value(from_to[0])
    # 2.2 st 월
    Select(driver.find_element(By.ID,"sSearchMonthFrom")).select_by_value(from_to[1])
    time.sleep(2)

     # 2.3 end 년 
    Select(driver.find_element(By.ID,"sSearchYearTo")).select_by_value(from_to[2])
    # 2.4 end 월
    Select(driver.find_element(By.ID,"sSearchMonthTo")).select_by_value(from_to[3])


In [63]:
monthly_start_date_list = [
    # ['2024', '01', '2024', '02'],
    # ['2024', '03', '2024', '04'],
    # ['2024', '05', '2024', '06'],
    # ['2024', '07', '2024', '08'],
    # ['2024', '09', '2024', '10'],
    # ['2024', '11', '2024', '12'],
    # ['2025', '01', '2025', '02'],
    # ['2025', '03', '2025', '04'],
    # ['2025', '05', '2025', '06'],
    # ['2025', '07', '2025', '08'],
    # ['2025', '09', '2025', '10']
    # ['2024', '01', '2024', '02'],
    # ['2024', '07', '2024', '08'],
    # ['2025', '01', '2025', '02'],
    # ['2025', '07', '2025', '08']
    # ['2024', '01', '2024', '02'],

    # ['2024', '04', '2024', '05'],
    # ['2024', '07', '2024', '08'],
    ['2024', '10', '2024', '11'],
    ['2025', '01', '2025', '02'],
    ['2025', '04', '2025', '05'],
    ['2025', '07', '2025', '08'],
    ['2025', '10', '2025', '11']
]

#region
#지역 선택: 개별지역 돌아가며 -> 전체로 한다.
region_select = driver.find_element(By.ID, "sWideAreaCd")
region_option_list = region_select.find_elements(By.TAG_NAME, "option")
region_list = [r_opt.get_attribute("value") for r_opt in region_option_list if r_opt.get_attribute("value")]

# driver.set_page_load_timeout(240)#2분 지나면 타임아웃 에러 발생!

#date
date_len = len(monthly_start_date_list)
for di, start_date in enumerate(monthly_start_date_list):
    print(di+1, '/', date_len, start_date, end='')
    set_monthly_searh_date_option_by_start_date(start_date)
    time.sleep(0.5)

    for r_val in tqdm(region_list[:3]):#!! 일부만!!
        print(r_val)
        region_select = driver.find_element(By.ID, "sWideAreaCd")# !!이렇게 안쓰니 StaleElementReferenceException 에러가 나는데...
        Select(region_select).select_by_value(r_val)
        time.sleep( random.uniform(1,2) )

        # clickSearchButton() # type: ignore 
        # # time.sleep( random.uniform(1,2) )

        # downloadExcelFile()
        # # time.sleep( random.uniform(3,5) )

    # clickSearchButton() # type: ignore # time.sleep( random.uniform(5,8) )

    # downloadExcelFile()
    # time.sleep( random.uniform(3,5) )


1 / 5 ['2024', '10', '2024', '11']

  0%|          | 0/3 [00:00<?, ?it/s]

0105001


 33%|███▎      | 1/3 [00:01<00:03,  1.77s/it]

0105002


...

 33%|███▎      | 1/3 [00:01<00:03,  1.72s/it]

0105002


 67%|██████▋   | 2/3 [00:03<00:01,  1.75s/it]

0105003


100%|██████████| 3/3 [00:04<00:00,  1.62s/it]


1. 문제
1.1 Read timeout 발생 : 2분 넘어가니 짤 없이 짤림. 서비스 제공하는 서버에서 Time out 2분 설정한 것 같음.
 - 검색조건 범위를 줄여서 2분 안으로 설정.
 ! 지역단위로, 2개월 간격 조회 => 월 단위 조회는 1개월 더 추가되어 조회됨. 수동으로 6개월치 조회했을 때에도 7개월치 조회되었음.
 => 24.1~2, 24.4~5, 24.7~8, 24.10~11 
    25.1~2, 25.4~5, 25.7~8, 25.10~11  이렇게 조회한다!!

2.1 간헐적인 StaleElementReferenceException 발생 cf) https://spiaminto.tistory.com/19
 - 셀레니움 드라이버가 페이지의 요소를 미리 기록해두었으나 이후의 요청 시점에 그 요소를 DOM 에서 엑세스 할 수 없을때 발생하는 예외이다. 
 주로 WebDriverWait.until() 로 찾아놓은 element 객체에 element.getAttribute() 메서드 호출시점에 발생했다. headless 모드를 끄고 브라우저를 통해 직접 확인해 보았으나 DOM 에 변화가 없었음에도 불구하고 예외가 발생했다.
 => try except 걸어, 해당 element 다시 찾아서 -> click 하는 걸로 수정. 아직 재현 안되어서... 체크 필요!


'여기까지. 24.1~3 월 17개 지역 월단위 xls 파일로 다운로드 oK!'

자, 이렇게 다시!
    ['2024', '04', '2024', '05'],
    ['2024', '07', '2024', '08'],
    ['2024', '10', '2024', '11'],
    ['2025', '01', '2025', '02'],
    ['2025', '04', '2025', '05'],
    ['2025', '07', '2025', '08'],
    ['2025', '10', '2025', '11']


---------------------------------------------------------------------------
StaleElementReferenceException            Traceback (most recent call last)
Cell In[41], line 50
     46     time.sleep( random.uniform(1,2) )
     48     clickSearchButton() # type: ignore # time.sleep( random.uniform(5,8) )
---> 50     downloadExcelFile()
     51     time.sleep( random.uniform(3,5) )
     53 # clickSearchButton() # type: ignore # time.sleep( random.uniform(5,8) )
     54 
     55 # downloadExcelFile()
     56 # time.sleep( random.uniform(3,5) )

Cell In[38], line 9
      6 print('Excel 다운로드!')
      7 global driver
----> 9 exl_btn = driver.find_element(By.CSS_SELECTOR,".f_sch").find_element(By.TAG_NAME, "a")
     10 try:
     11     WebDriverWait(driver, 240).until(EC.element_to_be_clickable(exl_btn))

File ~/venv/eda_venv/lib/python3.12/site-packages/selenium/webdriver/remote/webelement.py:602, in WebElement.find_element(self, by, value)
    576 """Find an element given a By strategy and locator.
    577 
    578 Parameters:
   (...)    599     The first matching `WebElement` found on the page.

월 기간 검색조건 selenium 으로 조작했을때, 
 24.10 - 24.11 설정시 => 24.06 - 24.11로 설정되어 2분 넘게 소요됨!
 25.10 - 25.11 설정시 같은문제 발생 안함!

 ==> 자바스크립트에 callback 함수로 기간 6개월로 자동 설정하도록 되어있는 것 같음. 보통 그렇게 사용함 - 이유는 .. 여러가지인데 생략.
 

---------------------------------------------------------------------------
TimeoutError                              Traceback (most recent call last)
File ~/venv/eda_venv/lib/python3.12/site-packages/urllib3/connectionpool.py:534, in HTTPConnectionPool._make_request(self, conn, method, url, body, headers, retries, timeout, chunked, response_conn, preload_content, decode_content, enforce_content_length)
    533 try:
--> 534     response = conn.getresponse()
    535 except (BaseSSLError, OSError) as e:

File ~/venv/eda_venv/lib/python3.12/site-packages/urllib3/connection.py:565, in HTTPConnection.getresponse(self)
    564 # Get the response from http.client.HTTPConnection
--> 565 httplib_response = super().getresponse()
    567 try:

File /usr/lib/python3.12/http/client.py:1428, in HTTPConnection.getresponse(self)
   1427 try:
-> 1428     response.begin()
   1429 except ConnectionError:

File /usr/lib/python3.12/http/client.py:331, in HTTPResponse.begin(self)
    330 while True:
--> 331     version, status, reason = self._read_status()
    332     if status != CONTINUE:

File /usr/lib/python3.12/http/client.py:292, in HTTPResponse._read_status(self)
    291 def _read_status(self):
--> 292     line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
...
    369     ) from err
    371 # See the above comment about EAGAIN in Python 3.
    372 if hasattr(err, "errno") and err.errno in _blocking_errnos:

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=59923): Read timed out. (read timeout=120)
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

In [10]:
driver.quit()